* Supongamos que tenemos **M** opciones mutuamente exclusivas:

  - **M** tratamientos diferentes con los que intentar tratar una nueva enfermedad.
  - **M** posibles formas de presentar una publicidad en el browser del usuario.

* **Máquina de Bernoulli:** el resultado (recompensa) de cada acción concreta es binario: 1 (éxito) o 0 (no éxito).

* La recompensa total es (número éxitos) / (número intentos).

* Cada una de las **M** opciones tendrá asociada una probabilidad de éxito diferente (**"payout"**):

  - Cada tratamiento tiene un porcentaje de éxito que a priori es desconocido.
  - Cada forma de publicidad tiene un porcentaje de éxito (probabilidad de que el usuario dé click sobre el anuncio) diferente y desconocida a priori.

* Problema: inicialmente no hay una base de datos de casos pasados: el agente de aprendizaje por refuerzo tiene que ir construyendo esta base de datos a la vez que intenta utilizarla de la manera más inteligente posible con el fin de maximizar la ganancia total:

  - Número de personas curadas
  - Número de clicks realizados sobre la publicidad ofrecida

* En este notebook experimentaremos con un problema con **M=10**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta

# np.random.beta(self.alphas, self.ns)
# pdf = beta(tsa.a[a], tsa.b[a]).pdf(x)

In [ ]:
x = np.linspace(0,1,1000)
n_exitos = 100
n_no_exitos = 100
pdf = beta(n_exitos+1, n_no_exitos+1).pdf(x) # primer argumento: número de éxitos + 1
# Segundo argumento: número de no éxitos + 1
plt.plot(x, pdf)
plt.xlabel("Q teórico")
plt.ylabel("densidad probabilidad")

In [ ]:
np.random.beta(n_exitos+1, n_no_exitos+1)

## Definir entorno

In [ ]:
class entorno():
  def __init__(self, payouts):
    self.payouts = payouts
    self.nk      = len(payouts) # número de máquinas
  def act(self, accion):
    return int(np.random.rand() <= self.payouts[accion])

In [ ]:
# probabilidad de premio en cada una de las 10 máquinas Bernoulli
# (dan premio o no, premio binario)

payouts = [0.8, 0.75, 0.7, 0.85, 0.9, 0.65, 0.6, 0.55, 0.5, 0.45]
e = entorno(payouts)

In [ ]:
rewards_maquina = [e.act(2) for _ in range(10000)]
np.mean(rewards_maquina)

## Agente $\epsilon$-Greedy:

In [ ]:
class EpsilonGreedy():
  def __init__(self, nk, epsilon):
    self.nk = nk
    self.epsilon = epsilon
    self.nveces = np.zeros(nk) # número de tiradas en las máquinas
    self.suma_rs = np.zeros(nk) # premios totales acumulados en las máquinas
    self.Qs = -10*np.ones(nk)

  # Implementación de la estrategia de decisión:
  def choose_action(self):
    if np.random.rand() < self.epsilon: # exploramos?
      return np.random.choice(self.nk)
    return np.argmax(self.Qs)

  def update(self, action, r): # con esta función el agente "aprende"
    self.nveces[action] += 1 # sumamos 1 al número de veces que se ha realizado la acción "action"
    self.suma_rs[action] += r # sumamos r a la recompensa total asociada a esa acción
    self.Qs[action] = self.suma_rs[action] / self.nveces[action] # recalculamos la Q

In [ ]:
def experimento(agente, e, N, nk):
  r_total = 0
  historico_r_total    = []
  historico_r_promedio = []
  historico_regret     = []
  historico_acciones   = []

  for i in range(N):
      action = agente.choose_action() # pregunto al agente en qué máquina introducir moneda
      reward = e.act(action) # introduzco moneda en esa máquina; reward es el refuerzo obtenido
      agente.update(action, reward) # el agente aprende con la acción elegida y el reward obtenido

      r_total += reward
      historico_r_total.append(r_total)
      historico_r_promedio.append(r_total/(i+1)) # i+1: número de monedas introducidas en total
      historico_regret.append(0.9*(i+1) - r_total) # yo sé que la tercera máquina es la mejor (premio promedio=3), porque la he programado así
      historico_acciones.append(action)

  # nos dibujamos alguna gráfica:

  plt.plot(historico_r_total)
  plt.title("Refuerzo total acumulado")
  plt.show()
  plt.plot(historico_r_promedio)
  plt.title("Refuerzo promedio")
  plt.show()
  plt.plot(historico_regret)
  plt.title("Regret total")
  print("Refuerzo total promedio final:", historico_r_promedio[-1])
  plt.show()
  historico_acciones = np.array(historico_acciones)
  for i in range(nk):
    aux = np.where(historico_acciones==i)[0] # en qué momentos de tiempo se elige la máquina i?
    plt.plot(aux, len(aux)*[i], ".", label="Máquina "+str(i)) # la máquina i la dibuja a altura (y) i
  plt.xlabel("Tiempo")
  plt.ylabel("Máquina")
  plt.legend()

  #print("Qs finales:", agente.Qs)

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

e      = entorno(payouts)
agente = EpsilonGreedy(epsilon=0.1, nk=10)
N = 10000 # número total de tiradas
experimento(agente, e, N, nk=10)

In [ ]:
payouts

## Agente bayesiano:

In [ ]:
class Bayesiano():
  def __init__(self, nk):
    self.nk = nk
    self.exitos    = np.zeros(nk) # número de tiradas en las máquinas
    self.no_exitos = np.zeros(nk) # número de tiradas en las máquinas
    self.nveces = np.zeros(nk) # número de tiradas en las máquinas

  # Implementación de la estrategia de decisión basada en el muestreo de Thompson:
  def choose_action(self):
    aux = np.random.beta(self.exitos+1, self.no_exitos+1) # obtengo un muestreo de Q teórica de cada una de las 10 máquinas
    return np.argmax(aux)

  def update(self, action, r): # con esta función el agente "aprende"
    self.nveces[action] += 1
    if r==1:
      self.exitos[action] += 1
    else:
      self.no_exitos[action] += 1

  def show_info(self):
    x = np.linspace(0,1,1000)
    for i in range(self.nk): # recorro los datos que tengo de cada una de las máquinas
      pdf = beta(self.exitos[i]+1, self.no_exitos[i]+1).pdf(x) # primer argumento: número de éxitos + 1
      plt.plot(x, pdf, label="Máquina "+str(i))
    plt.xlabel("Q teórico")
    plt.ylabel("densidad probabilidad")
    plt.legend()

In [ ]:
agente = Bayesiano(nk=10)

In [ ]:
agente.show_info()

In [ ]:
maquina = 2
reward = 1
agente.update(maquina, reward)
agente.show_info()

In [ ]:
# Defino entorno y agente desde 0, y simulo aprendizaje:

e      = entorno(payouts)
agente = Bayesiano(nk=10)
N = 100000 # número total de tiradas
experimento(agente, e, N, nk=10)

In [ ]:
agente.show_info()

In [ ]:
payouts